
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://cct.ued.vnu.edu.vn/admin//lienketlinkphai/brtunv-7_87_anhlienket.png" alt="Databricks Learning" style="width: 600px">
</div>


Các bạn sẽ sử dụng Databricks để lấy dữ liệu từ bảng chi tiết chuyến đi của của một hãng taxi tại thành phố New York `samples.nyctaxi.trips` có sẵn trên Databricks. 
Lưu ý: Bảng sẽ xuất hiện sau khi bật cluster.

Thông tin các trường và ý nghĩa như sau:

| Tên trường | Ý nghĩa |
| --- | --- |
| tpep_pickup_datetime | Thời gian đón khách (chính xác đến milisecond) |
| tpep_dropoff_datetime | Thời gian trả khách (chính xác đến milisecond) |
| trip_distance | Quãng đường chuyến đi (đơn vị: dặm - miles) |
| fare_amount | Số tiền (đơn vị: $) |
| pickup_zip | Mã điểm đón khách |
| dropoff_zip | Mã điểm trả khách |


In [0]:
val input = spark.read.table("samples.nyctaxi.trips")

input: org.apache.spark.sql.DataFrame = [tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp ... 4 more fields]

In [0]:
display(input.limit(10))

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-14T16:52:13.000+0000,2016-02-14T17:16:04.000+0000,4.94,19.0,10282,10171
2016-02-04T18:44:19.000+0000,2016-02-04T18:46:00.000+0000,0.28,3.5,10110,10110
2016-02-17T17:13:57.000+0000,2016-02-17T17:17:55.000+0000,0.7,5.0,10103,10023
2016-02-18T10:36:07.000+0000,2016-02-18T10:41:45.000+0000,0.8,6.0,10022,10017
2016-02-22T14:14:41.000+0000,2016-02-22T14:31:52.000+0000,4.51,17.0,10110,10282
2016-02-05T06:45:02.000+0000,2016-02-05T06:50:26.000+0000,1.8,7.0,10009,10065
2016-02-15T15:03:28.000+0000,2016-02-15T15:18:45.000+0000,2.58,12.0,10153,10199
2016-02-25T19:09:26.000+0000,2016-02-25T19:24:50.000+0000,1.4,11.0,10112,10069
2016-02-13T16:28:18.000+0000,2016-02-13T16:36:36.000+0000,1.21,7.5,10023,10153
2016-02-14T00:03:48.000+0000,2016-02-14T00:10:24.000+0000,0.6,6.0,10012,10003


Bài 1: Hãy tính tiền trung bình của các chuyến đi có quãng đường đi lớn hơn 2 miles

In [0]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._

In [0]:
val lesson1 = input.filter($"trip_distance" > 2).agg(round(avg($"fare_amount"), 5).as("AVG_AMOUNT"))
display(lesson1)

AVG_AMOUNT
19.7202


Bài 2: Hãy cho biết thời gian trung bình của mỗi chuyến đi trên tổng số chuyến đi đã thực hiện

In [0]:
input.printSchema()

root
-- tpep_pickup_datetime: timestamp (nullable = true)
-- tpep_dropoff_datetime: timestamp (nullable = true)
-- trip_distance: double (nullable = true)
-- fare_amount: double (nullable = true)
-- pickup_zip: integer (nullable = true)
-- dropoff_zip: integer (nullable = true)

In [0]:
val lesson2 = input.withColumn("new_time", unix_timestamp($"tpep_dropoff_datetime") - unix_timestamp($"tpep_pickup_datetime"))
                   .agg(avg($"new_time")) //unix_timestamp là kết quả trả về giây.
display(lesson2)

avg(new_time)
906.9607878898413


Bài 3: Hãy tìm cặp zip code (`pickup_zip`, `dropoff_zip`) có quãng đường trung bình lớn nhất

In [0]:
val lesson3 = input.orderBy(desc("trip_distance")).select($"pickup_zip", $"dropoff_zip", $"trip_distance").limit(1)
display(lesson3)

pickup_zip,dropoff_zip,trip_distance
11371,7114,30.6


Bài 4: Hãy hiển thị top 3 bản ghi có giá tiền lớn nhất theo mỗi cặp `pickup_zip`, `dropoff_zip`

In [0]:
// Kiểm tra các cặp pickup dropoff
val test = input.groupBy($"pickup_zip", $"dropoff_zip").count.orderBy(desc("count")).select($"pickup_zip", $"dropoff_zip").limit(3)
display(test)

pickup_zip,dropoff_zip
10023,10023
10028,10028
10028,10021


In [0]:
import org.apache.spark.sql.expressions.Window
val w = Window.orderBy(desc("max_amount"))
val lesson4 = input.groupBy($"pickup_zip", $"dropoff_zip")
                   .agg(count("*").as("max_amount"))
                   .withColumn("top3", row_number().over(w))
                   .filter($"top3" <=3)
                   .drop("top3")
                   .join(input, Seq("pickup_zip", "dropoff_zip"))
                   .groupBy($"pickup_zip", $"dropoff_zip")
                   .agg(max($"fare_amount").as("max_fare_amount"))
display(lesson4)

pickup_zip,dropoff_zip,max_fare_amount
10028,10028,52.0
10028,10021,13.0
10023,10023,52.0


In [0]:
// Cách 2 làm tuần tự
val top3 = input.groupBy($"pickup_zip", $"dropoff_zip")
                .agg(count("*").as("count"))

val w = Window.orderBy(desc("count"))
val top_pick_drop = top3.withColumn("top_3", row_number().over(w)).filter($"top_3" <= 3).drop("top_3") // cần xóa top_3 vì chỉ là giá trị trung gian để lọc lấy top 3 giá trị count lớn nhất

val result = input.join(top_pick_drop, Seq("pickup_zip", "dropoff_zip")) // dùng Seq để lọc và giữ lại những dòng ở input có cùng giá trị pickup và dropoff ở bảng top_pick_drop
                  .groupBy($"pickup_zip", $"dropoff_zip")
                  .agg(max($"fare_amount").as("max_fare_amount"))

display(result)

pickup_zip,dropoff_zip,max_fare_amount
10028,10028,52.0
10028,10021,13.0
10023,10023,52.0


Bài 5: Hãy cho biết số tiền kiếm được của tất cả chuyến đi theo ngày trong tuần. Kết quả mẫu sẽ như sau:

| Date_of_week | amount |
| --- | --- |
| Monday | 10.000 |
| Tuesday | 20.000 |
| ... | ...

In [0]:
val lesson5 = input.withColumn("Date_of_week", date_format($"tpep_dropoff_datetime", "EEEE"))
                   .groupBy($"Date_of_week")
                   .agg(round(sum($"fare_amount"), 4))
display(lesson5)

Date_of_week,"round(sum(fare_amount), 4)"
Wednesday,36602.6
Tuesday,34094.5
Friday,44542.01
Thursday,40820.02
Saturday,40136.5
Monday,37198.8
Sunday,37437.85


Sử dụng dữ liệu tại file: `dbfs:/FileStore/nyc_trips/FileNYC_Zipcode_Name.csv`. Thực hiện Join dữ liệu file với bảng `samples.nyctaxi.trips` theo trường `Zipcode` của CSV và trường `pickup_zip` của bảng

In [0]:
val zip_name = spark.read.option("header", "true").csv("dbfs:/FileStore/nyc_trips/FileNYC_Zipcode_Name.csv")

zip_name: org.apache.spark.sql.DataFrame = [Borough: string, Neighborhood: string ... 1 more field]

In [0]:
display(zip_name.limit(10))

Borough,Neighborhood,ZipCode
Bronx,Central Bronx,10453
Bronx,Central Bronx,10457
Bronx,Central Bronx,10460
Bronx,Bronx Park and Fordham,10458
Bronx,Bronx Park and Fordham,10467
Bronx,Bronx Park and Fordham,10468
Bronx,High Bridge and Morrisania,10451
Bronx,High Bridge and Morrisania,10452
Bronx,High Bridge and Morrisania,10456
Bronx,Hunts Point and Mott Haven,10454


Bài 6: Hãy cho biết Doanh thu của các quận trong thành phố với format như sau:

| Tên trường | Ý nghĩa |
| --- | --- |
| borough | Tên quận (Brooklyn, Manhattan, ...) |
| amount | Doanh thu |


In [0]:
val lesson6 = input.join(zip_name, $"pickup_zip" === $"ZipCode", "inner")
                   .groupBy($"Borough")
                   .agg(sum($"fare_amount").as("amount"))
display(lesson6)

Borough,amount
Queens,23193.5
Brooklyn,5453.0
Manhattan,168325.62
Bronx,172.51
Staten Island,13.5
